In [1]:
import numpy as np 
import pandas as pd
import geopandas as gpd

import rasterio
from rasterio.plot import reshape_as_image
from rasterio.features import rasterize

from shapely.geometry import mapping, Point, Polygon
from shapely.ops import unary_union

import matplotlib.pyplot as plt

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("isaienkov/deforestation-in-ukraine")

print("Path to dataset files:", path)

C:\Users\Pele\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.13_qbz5n2kfra8p0\LocalCache\local-packages\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


  0%|          | 9.00M/32.7G [00:01<1:32:24, 6.33MB/s]


KeyboardInterrupt: 

In [ ]:
path = "/home/epele/.cache/kagglehub/datasets/isaienkov/deforestation-in-ukraine/versions/1"
df = gpd.read_file(path + "/deforestation_labels.geojson")
print("Number of polygons:", len(df))
df.head()

In [ ]:
RASTER_PATH = path + '/S2B_MSIL1C_20190611T083609_N0207_R064_T36UYA_20190611T122426/S2B_MSIL1C_20190611T083609_N0207_R064_T36UYA_20190611T122426.SAFE/GRANULE/L1C_T36UYA_A011816_20190611T084501/IMG_DATA/T36UYA_20190611T083609_TCI.jp2'

with rasterio.open(RASTER_PATH, "r", driver='JP2OpenJPEG') as src:
    raster_image = src.read()
    raster_meta = src.meta

In [ ]:
raster_image.shape
raster_meta

In [ ]:
raster_img = reshape_as_image(raster_image)
plt.figure(figsize=(10, 10))
plt.imshow(raster_img)

In [ ]:
# assigning crs
# use 4236 for tiles from this dataset

df

In [ ]:
from rasterio.features import rasterize

RASTER_PATH = path + '/S2B_MSIL1C_20190611T083609_N0207_R064_T36UYA_20190611T122426/S2B_MSIL1C_20190611T083609_N0207_R064_T36UYA_20190611T122426.SAFE/GRANULE/L1C_T36UYA_A011816_20190611T084501/IMG_DATA/T36UYA_20190611T083609_TCI.jp2'

with rasterio.open(RASTER_PATH, "r", driver='JP2OpenJPEG') as src:
    raster_image = src.read()
    raster_meta = src.meta

    raster_img = reshape_as_image(raster_image)

    df.crs = {'init':'epsg:4236'}

    #transforming polygons to the raster crs
    df = df.to_crs(raster_meta['crs'])
    # `src` is your opened Sentinel-2 raster

    shapes = [(geom, 1) for geom in df.geometry]
    mask = rasterize(
        shapes,
        out_shape=(src.height, src.width),
        transform=src.transform,
        fill=0,
        dtype='uint8'
    )
    # mask now is a (H,W) array of 0/1
    plt.figure(figsize=(12,12))
    plt.imshow(mask)

In [ ]:
import rasterio
from rasterio.features import rasterize

# Open one example Sentinel-2 tile to get metadata
with rasterio.open(RASTER_PATH, "r", driver='JP2OpenJPEG') as src:
    transform = src.transform
    height, width = src.height, src.width

# Prepare shapes as (geometry, value) pairs
shapes = [(geom, 1) for geom in df.geometry]

# Rasterize to binary mask
mask = rasterize(
    shapes,
    out_shape=(height, width),
    transform=transform,
    fill=0,
    dtype='uint8'
)

In [ ]:
import numpy as np

with rasterio.open(RASTER_PATH) as src:
    # Example for 3 bands; adjust indexes for your dataset
    red   = src.read(1)
    green = src.read(2)
    blue  = src.read(3)

# Stack into (H, W, 3)
image = np.stack([red, green, blue], axis=-1).astype('float32') / 10000.0

In [ ]:
df.head()

# Preprocess the data

In [ ]:
folders_path = '/home/epele/.cache/kagglehub/datasets/isaienkov/deforestation-in-ukraine/versions/1/'

df = gpd.read_file(path + "/deforestation_labels.geojson")
print("Number of polygons:", len(df))
df.head()

# MODEL

In [ ]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Conv2D, Conv2DTranspose, Activation,
    MaxPooling2D, UpSampling2D,
    add, multiply, concatenate
)
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Activation, UpSampling2D,
    add, multiply, Lambda
)

'''
  Convolutional block with set parameters and activation layer after
'''

def convBlock(input, filters, kernel, kernel_init='he_normal', act='relu', transpose=False):
  if transpose == False:
    #conv = ZeroPadding2D((1,1))(input)
    conv = Conv2D(filters, kernel, padding = 'same', kernel_initializer = kernel_init)(input)
  else:
    #conv = ZeroPadding2D((1,1))(input)
    conv = Conv2DTranspose(filters, kernel, padding = 'same', kernel_initializer = kernel_init)(input)

  conv = Activation(act)(conv)
  return conv

'''
  Convolutional block with two conv layers and two activation layers
'''

def convBlock2(input, filters, kernel, kernel_init='he_normal', act='relu', transpose=False):
  if transpose == False:
    conv = Conv2D(filters, kernel, padding = 'same', kernel_initializer = kernel_init)(input)
    conv = Activation(act)(conv)
    conv = Conv2D(filters, kernel, padding = 'same', kernel_initializer = kernel_init)(conv)
    conv = Activation(act)(conv)
  else:
    conv = Conv2DTranspose(filters, kernel, padding = 'same', kernel_initializer = kernel_init)(input)
    conv = Activation(act)(conv)
    conv = Conv2DTranspose(filters, kernel, padding = 'same', kernel_initializer = kernel_init)(conv)
    conv = Activation(act)(conv)

  return conv
  
'''
  Attention block/mechanism
'''
def attention_block(x, gating, inter_shape, drop_rate=0.25):
    shape_x = K.int_shape(x)
    # 1x1 conv + pool on x
    theta_x = Conv2D(inter_shape, 1, padding='same')(x)
    theta_x = MaxPooling2D((2,2))(theta_x)

    # 1x1 conv on gating signal
    phi_g = Conv2D(inter_shape, 1, padding='same')(gating)

    act_xg = Activation('relu')(add([phi_g, theta_x]))
    psi    = Conv2D(1, 1, padding='same')(act_xg)
    sigmoid_xg = Activation('sigmoid')(psi)

    # Upsample back to x’s spatial dims
    upsample_psi = UpSampling2D(
        interpolation='bilinear',
        size=(
            shape_x[1] // K.int_shape(sigmoid_xg)[1],
            shape_x[2] // K.int_shape(sigmoid_xg)[2]
        )
    )(sigmoid_xg)

    # Wrap broadcast inside a Keras layer
    upsample_psi = Lambda(
        lambda t: tf.broadcast_to(t, shape_x)
    )(upsample_psi)

    # Elementwise multiplication (broadcasting mask over channels)
    y = multiply([upsample_psi, x])
    return y


'''
  Attention U-Net model
'''

def UNetAM(trained_weights = None, input_size = (512,512,3), drop_rate = 0.25, lr=0.0001, filter_base=16):

    ## Can add pretrained weights by specifying 'trained_weights'

    # Input layer
    inputs = Input(input_size, batch_size=1)

    ## Contraction phase
    conv = convBlock2(inputs, filter_base, 3)
    #conv0 = Dropout(drop_rate)(conv0)

    conv0 = MaxPooling2D(pool_size=(2, 2))(conv)
    conv0 = convBlock2(conv0, 2 * filter_base, 3)

    pool0 = MaxPooling2D(pool_size=(2, 2))(conv0)
    conv1 = convBlock2(pool0, 4 * filter_base, 3)
    #conv1 = Dropout(drop_rate)(conv1)

    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = convBlock2(pool1, 8 * filter_base, 3)
    #conv2 = Dropout(drop_rate)(conv2)

    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = convBlock2(pool2, 16 * filter_base, 3)
    #conv3 = Dropout(drop_rate)(conv3)

    ## Expansion phase
    up4 = (Conv2DTranspose(8 * filter_base, kernel_size=2, strides=2, kernel_initializer='he_normal')(conv3))
    merge4 = attention_block(conv2, conv3, 8 * filter_base, drop_rate) # Attention gate
    conv4 = concatenate([up4, merge4])
    conv4 = convBlock2(conv4, 8 * filter_base, 3)

    up5 = (Conv2DTranspose(4 * filter_base, kernel_size=2, strides=2, kernel_initializer='he_normal')(conv4))
    merge5 = attention_block(conv1, conv4, 4 * filter_base, drop_rate) # Attention gate
    conv5 = concatenate([up5, merge5])
    conv5 = convBlock2(conv5, 4 * filter_base, 3)

    up6 = (Conv2DTranspose(2 * filter_base, kernel_size=2, strides=2, kernel_initializer='he_normal')(conv5))
    merge6 = attention_block(conv0, conv5, 2 * filter_base, drop_rate) # Attention gate
    conv6 = concatenate([up6, merge6])
    conv6 = convBlock2(conv6, 2 * filter_base, 3)

    up7 = (Conv2DTranspose(1 * filter_base, kernel_size=2, strides=2, kernel_initializer='he_normal')(conv6))
    merge7 = attention_block(conv, conv6, 1 * filter_base, drop_rate) # Attention gate
    conv7 = concatenate([up7, merge7])
    conv7 = concatenate([up7, conv])
    conv7 = convBlock2(conv7, 1 * filter_base, 3)

    ## Output layer
    out = convBlock(conv7, 1, 1, act='sigmoid')

    model = Model(inputs, out)

    model.compile(optimizer = Adam(learning_rate = lr), loss = binary_crossentropy, metrics = ['accuracy', 'mse'])

    if trained_weights != None:
    	model.load_weights(trained_weights)

    return model

In [ ]:
model = UNetAM()
model.summary()